# 論文 14：透過聯合學習對齊與翻譯的神經機器翻譯
## Dzmitry Bahdanau, KyungHyun Cho, Yoshua Bengio (2014)

### 原始注意力機制

這篇論文引入了**注意力（Attention）**機制——深度學習中最重要的創新之一。它比 Transformers 早了 3 年！

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

## 問題：固定長度的上下文向量

傳統的 seq2seq 將整個輸入壓縮成單一向量 → 資訊瓶頸！

In [ ]:
def softmax(x, axis=-1):
    exp_x = np.exp(x - np.max(x, axis=axis, keepdims=True))
    return exp_x / np.sum(exp_x, axis=axis, keepdims=True)

class EncoderRNN:
    """雙向 RNN 編碼器"""
    def __init__(self, input_size, hidden_size):
        self.hidden_size = hidden_size
        
        # 前向 RNN
        self.W_fwd = np.random.randn(hidden_size, input_size + hidden_size) * 0.01
        self.b_fwd = np.zeros((hidden_size, 1))
        
        # 後向 RNN
        self.W_bwd = np.random.randn(hidden_size, input_size + hidden_size) * 0.01
        self.b_bwd = np.zeros((hidden_size, 1))
    
    def forward(self, inputs):
        """
        inputs：(input_size, 1) 向量的列表
        返回：雙向隱藏狀態的列表 (2*hidden_size, 1)
        """
        seq_len = len(inputs)
        
        # 前向傳遞
        h_fwd = []
        h = np.zeros((self.hidden_size, 1))
        for x in inputs:
            concat = np.vstack([x, h])
            h = np.tanh(np.dot(self.W_fwd, concat) + self.b_fwd)
            h_fwd.append(h)
        
        # 後向傳遞
        h_bwd = []
        h = np.zeros((self.hidden_size, 1))
        for x in reversed(inputs):
            concat = np.vstack([x, h])
            h = np.tanh(np.dot(self.W_bwd, concat) + self.b_bwd)
            h_bwd.append(h)
        h_bwd = list(reversed(h_bwd))
        
        # 串接前向和後向
        annotations = [np.vstack([h_f, h_b]) for h_f, h_b in zip(h_fwd, h_bwd)]
        
        return annotations

print("雙向編碼器已建立")

## Bahdanau 注意力機制

關鍵創新：聯合對齊與翻譯！

**注意力分數**：$e_{ij} = a(s_{i-1}, h_j)$，其中 $s$ 是解碼器狀態，$h$ 是編碼器標註

**注意力權重**：$\alpha_{ij} = \frac{\exp(e_{ij})}{\sum_k \exp(e_{ik})}$

**上下文向量**：$c_i = \sum_j \alpha_{ij} h_j$

In [ ]:
class BahdanauAttention:
    """加性注意力機制"""
    def __init__(self, hidden_size, annotation_size):
        self.hidden_size = hidden_size
        
        # 注意力參數
        self.W_a = np.random.randn(hidden_size, hidden_size) * 0.01
        self.U_a = np.random.randn(hidden_size, annotation_size) * 0.01
        self.v_a = np.random.randn(1, hidden_size) * 0.01
    
    def forward(self, decoder_hidden, encoder_annotations):
        """
        decoder_hidden：(hidden_size, 1) - 當前解碼器狀態 s_{i-1}
        encoder_annotations：(annotation_size, 1) 的列表 - 所有編碼器狀態 h_j
        
        返回：
        context：(annotation_size, 1) - 標註的加權和
        attention_weights：(seq_len,) - 注意力分佈
        """
        scores = []
        
        # 計算每個位置的注意力分數
        for h_j in encoder_annotations:
            # e_ij = v_a^T * tanh(W_a * s_{i-1} + U_a * h_j)
            score = np.dot(self.v_a, np.tanh(
                np.dot(self.W_a, decoder_hidden) + 
                np.dot(self.U_a, h_j)
            ))
            scores.append(score[0, 0])
        
        # Softmax 得到注意力權重
        scores = np.array(scores)
        attention_weights = softmax(scores)
        
        # 計算上下文向量作為加權和
        context = sum(alpha * h for alpha, h in zip(attention_weights, encoder_annotations))
        
        return context, attention_weights

print("Bahdanau 注意力機制已建立")

## 帶注意力的解碼器

In [ ]:
class AttentionDecoder:
    """帶 Bahdanau 注意力的 RNN 解碼器"""
    def __init__(self, output_size, hidden_size, annotation_size):
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # 注意力機制
        self.attention = BahdanauAttention(hidden_size, annotation_size)
        
        # RNN：接收前一個輸出 + 上下文
        input_size = output_size + annotation_size
        self.W_dec = np.random.randn(hidden_size, input_size + hidden_size) * 0.01
        self.b_dec = np.zeros((hidden_size, 1))
        
        # 輸出層
        self.W_out = np.random.randn(output_size, hidden_size + annotation_size + output_size) * 0.01
        self.b_out = np.zeros((output_size, 1))
    
    def step(self, prev_output, decoder_hidden, encoder_annotations):
        """
        單一解碼步驟
        
        prev_output：(output_size, 1) - 前一個輸出詞
        decoder_hidden：(hidden_size, 1) - 前一個解碼器狀態
        encoder_annotations：(annotation_size, 1) 的列表 - 編碼器狀態
        
        返回：
        output：(output_size, 1) - 預測的輸出分佈
        new_hidden：(hidden_size, 1) - 新的解碼器狀態
        attention_weights：注意力分佈
        """
        # 計算注意力和上下文
        context, attention_weights = self.attention.forward(decoder_hidden, encoder_annotations)
        
        # 解碼器 RNN：s_i = f(s_{i-1}, y_{i-1}, c_i)
        rnn_input = np.vstack([prev_output, context])
        concat = np.vstack([rnn_input, decoder_hidden])
        new_hidden = np.tanh(np.dot(self.W_dec, concat) + self.b_dec)
        
        # 輸出：y_i = g(s_i, y_{i-1}, c_i)
        output_input = np.vstack([new_hidden, context, prev_output])
        output = np.dot(self.W_out, output_input) + self.b_out
        
        return output, new_hidden, attention_weights
    
    def forward(self, encoder_annotations, max_length=20, start_token=None):
        """
        完整解碼
        """
        if start_token is None:
            start_token = np.zeros((self.output_size, 1))
        
        outputs = []
        attention_history = []
        
        # 初始化
        decoder_hidden = np.zeros((self.hidden_size, 1))
        prev_output = start_token
        
        for _ in range(max_length):
            output, decoder_hidden, attention_weights = self.step(
                prev_output, decoder_hidden, encoder_annotations
            )
            
            outputs.append(output)
            attention_history.append(attention_weights)
            
            # 下一個輸入是當前輸出（貪婪解碼）
            prev_output = output
        
        return outputs, attention_history

print("注意力解碼器已建立")

## 完整的帶注意力 Seq2Seq 模型

In [ ]:
class Seq2SeqWithAttention:
    def __init__(self, input_vocab_size, output_vocab_size, hidden_size=32):
        self.input_vocab_size = input_vocab_size
        self.output_vocab_size = output_vocab_size
        self.hidden_size = hidden_size
        
        # 嵌入層
        self.input_embedding = np.random.randn(input_vocab_size, hidden_size) * 0.01
        self.output_embedding = np.random.randn(output_vocab_size, hidden_size) * 0.01
        
        # 編碼器（雙向，所以標註大小是 2*hidden_size）
        self.encoder = EncoderRNN(hidden_size, hidden_size)
        
        # 帶注意力的解碼器
        annotation_size = 2 * hidden_size
        self.decoder = AttentionDecoder(hidden_size, hidden_size, annotation_size)
    
    def translate(self, input_sequence, max_output_length=15):
        """
        將輸入序列翻譯成輸出序列
        
        input_sequence：token 索引的列表
        """
        # 嵌入輸入
        embedded = [self.input_embedding[idx:idx+1].T for idx in input_sequence]
        
        # 編碼
        annotations = self.encoder.forward(embedded)
        
        # 解碼
        start_token = self.output_embedding[0:1].T  # 使用第一個 token 作為開始
        outputs, attention_history = self.decoder.forward(
            annotations, max_length=max_output_length, start_token=start_token
        )
        
        return outputs, attention_history, annotations

# 建立模型
input_vocab_size = 20   # 源語言詞彙表
output_vocab_size = 20  # 目標語言詞彙表
model = Seq2SeqWithAttention(input_vocab_size, output_vocab_size, hidden_size=16)

print(f"帶注意力的 Seq2Seq 已建立")
print(f"輸入詞彙表：{input_vocab_size}")
print(f"輸出詞彙表：{output_vocab_size}")

## 在合成翻譯任務上測試

In [ ]:
# 簡單的合成任務：反轉序列
# 輸入：[1, 2, 3, 4, 5]
# 輸出：[5, 4, 3, 2, 1]

input_seq = [1, 2, 3, 4, 5, 6, 7]
outputs, attention_history, annotations = model.translate(input_seq, max_output_length=len(input_seq))

print(f"輸入序列：{input_seq}")
print(f"輸出步數：{len(outputs)}")
print(f"注意力分佈數：{len(attention_history)}")
print(f"編碼器標註形狀：{len(annotations)} x {annotations[0].shape}")

## 視覺化注意力權重

關鍵洞察：看看模型注意什麼！

In [ ]:
# 將注意力歷史轉換為矩陣
attention_matrix = np.array(attention_history)  # (output_len, input_len)

plt.figure(figsize=(10, 8))
plt.imshow(attention_matrix, cmap='Blues', aspect='auto', interpolation='nearest')
plt.colorbar(label='注意力權重')
plt.xlabel('輸入位置（源）')
plt.ylabel('輸出位置（目標）')
plt.title('Bahdanau 注意力對齊矩陣')

# 添加網格
plt.xticks(range(len(input_seq)), [f'x{i+1}' for i in input_seq])
plt.yticks(range(len(outputs)), [f'y{i+1}' for i in range(len(outputs))])

plt.tight_layout()
plt.show()

print("\n注意力模式顯示哪些輸入位置影響每個輸出。")
print("較亮的格子 = 較高的注意力權重。")

## 每個解碼器步驟的注意力

In [ ]:
# 視覺化特定解碼器步驟的注意力分佈
fig, axes = plt.subplots(2, 4, figsize=(16, 6))
axes = axes.flatten()

steps_to_show = min(8, len(attention_history))

for i in range(steps_to_show):
    axes[i].bar(range(len(input_seq)), attention_history[i])
    axes[i].set_title(f'輸出步驟 {i+1}')
    axes[i].set_xlabel('輸入位置')
    axes[i].set_ylabel('注意力權重')
    axes[i].set_ylim(0, 1)
    axes[i].set_xticks(range(len(input_seq)))
    axes[i].set_xticklabels([f'x{j+1}' for j in input_seq], fontsize=8)
    axes[i].grid(True, alpha=0.3, axis='y')

plt.suptitle('每個解碼步驟的注意力分佈', fontsize=14)
plt.tight_layout()
plt.show()

print("每個解碼器步驟關注不同的輸入位置！")

## 比較：有注意力 vs 無注意力

In [ ]:
# 模擬固定上下文的 seq2seq（無注意力）
def fixed_context_attention(seq_len):
    """模擬只注意最後一個編碼器狀態"""
    weights = np.zeros(seq_len)
    weights[-1] = 1.0  # 只注意最後一個位置
    return weights

# 建立比較
input_length = len(input_seq)
output_length = len(outputs)

# 固定上下文
fixed_attention = np.array([fixed_context_attention(input_length) for _ in range(output_length)])

# 繪製比較圖
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 無注意力（固定上下文）
im1 = ax1.imshow(fixed_attention, cmap='Blues', aspect='auto', vmin=0, vmax=1)
ax1.set_xlabel('輸入位置')
ax1.set_ylabel('輸出位置')
ax1.set_title('無注意力（固定上下文）\n所有解碼器步驟只看最後一個編碼器狀態')
plt.colorbar(im1, ax=ax1)

# 有 Bahdanau 注意力
im2 = ax2.imshow(attention_matrix, cmap='Blues', aspect='auto', vmin=0, vmax=1)
ax2.set_xlabel('輸入位置')
ax2.set_ylabel('輸出位置')
ax2.set_title('有 Bahdanau 注意力\n每個解碼器步驟注意不同位置')
plt.colorbar(im2, ax=ax2)

plt.tight_layout()
plt.show()

print("\n關鍵差異：")
print("  無注意力：資訊瓶頸在最後一個編碼器狀態")
print("  有注意力：動態存取所有編碼器狀態")

## 注意力機制的變體

In [ ]:
def bahdanau_score(s, h, W_a, U_a, v_a):
    """加性/串接注意力（Bahdanau）"""
    return np.dot(v_a.T, np.tanh(np.dot(W_a, s) + np.dot(U_a, h)))[0, 0]

def dot_product_score(s, h):
    """點積注意力（Luong）"""
    return np.dot(s.T, h)[0, 0]

def scaled_dot_product_score(s, h):
    """縮放點積（Transformer 風格）"""
    d_k = s.shape[0]
    return np.dot(s.T, h)[0, 0] / np.sqrt(d_k)

# 比較評分函數
s = np.random.randn(16, 1)
h = np.random.randn(32, 1)
W_a = np.random.randn(16, 16)
U_a = np.random.randn(16, 32)
v_a = np.random.randn(1, 16)

print("注意力評分函數：")
print(f"  Bahdanau（加性）：score = v^T tanh(W*s + U*h)")
print(f"  點積：score = s^T h")
print(f"  縮放點積：score = s^T h / sqrt(d_k)")
print(f"\nBahdanau 更具表達力但有更多參數。")

## 關鍵要點

### 注意力解決的問題：
- **固定長度上下文**：整個輸入壓縮成單一向量
- **資訊瓶頸**：長序列會丟失資訊
- **無對齊**：解碼器不知道該關注哪個輸入

### Bahdanau 注意力的創新：
1. **動態上下文**：每個解碼器步驟不同
2. **軟對齊**：學習對齊源和目標
3. **所有編碼器狀態**：解碼器可存取所有狀態，而不只是最後一個

### 運作方式：
```
1. 編碼器產生標註 h_1, ..., h_T
2. 對於每個解碼器步驟 i：
   a. 計算注意力分數：e_ij = score(s_{i-1}, h_j)
   b. 正規化為權重：α_ij = softmax(e_ij)
   c. 計算上下文：c_i = Σ α_ij * h_j
   d. 生成輸出：y_i = f(s_i, c_i, y_{i-1})
```

### Bahdanau vs Luong 注意力：
| 特性 | Bahdanau (2014) | Luong (2015) |
|------|----------------|---------------|
| 分數 | 加性：v·tanh(W·s + U·h) | 乘性：s·h |
| 時機 | 使用 s_{i-1}（前一個） | 使用 s_i（當前） |
| 全域/局部 | 僅全域 | 兩者都有 |

### 數學公式：

**注意力分數（對齊模型）**：
$$e_{ij} = v_a^T \tanh(W_a s_{i-1} + U_a h_j)$$

**注意力權重**：
$$\alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k=1}^{T_x} \exp(e_{ik})}$$

**上下文向量**：
$$c_i = \sum_{j=1}^{T_x} \alpha_{ij} h_j$$

**解碼器**：
$$s_i = f(s_{i-1}, y_{i-1}, c_i)$$
$$p(y_i | y_{<i}, x) = g(s_i, y_{i-1}, c_i)$$

### 影響：
- **革新 NMT**：BLEU 分數大幅躍升
- **可解釋性**：可以視覺化對齊
- **Transformers 的基礎**：純注意力（2017）
- **超越 NMT**：用於視覺、語音等

### 為什麼有效：
1. **解決瓶頸**：可變長度上下文
2. **學習對齊**：不需要獨立的對齊模型
3. **可微分**：端到端訓練
4. **適用長序列**：注意力不會衰減

### 現代觀點：
- Transformers 使用**自注意力**（注意同一序列）
- 縮放點積現在是標準（更簡單、更快）
- 多頭注意力捕捉不同的關係
- 但 Bahdanau 的核心思想依然存在：**注意相關的東西**